<a href="https://colab.research.google.com/github/cbi-automation/lk-extraction/blob/fitriadc-dev/MVP_FinX_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📄 FinX : Tools Ekstraksi Informasi Laporan Keuangan

Program ini dibuat untuk mengekstraksi informasi penting dari laporan keuangan (PDF) berbagai perusahaan yang terdaftar di Bursa Efek Indonesia. Dokumentasi ini ditujukan untuk pembaca non-teknis maupun teknis, agar dapat memahami alur logika program dan mengembangkannya lebih lanjut.

# Product Documentation

## System Documentation

### **Arsitektur Sistem**

FinX tersusun dari beberapa modul utama:

- `Main.ipynb`: Entry point untuk menjalankan proses ekstraksi.
- `marker_config.py`: Marker unik per emiten yang memandu proses ekstraksi.

### **Alur Data**

- **Input**: Laporan keuangan (PDF) dari situs BEI.
- **Proses**: Dibaca dengan `pdfplumber`, dipilah berdasarkan marker.
- **Output**: File `.xlsx` berisi informasi yang terstruktur.

## User Documentation

### **Instalasi Program**

- Tambahkan pintasan folder berikut ke Google Drive lokal (root):
   [📁 Folder Drive FinX](https://drive.google.com/drive/folders/1mZBcnBDgAkWPbhptWDl6UhLeUB6_ps6M)

- Buka link program berikut:
   [🔗 Program Utama FinX (Colab)](https://github.com/cbi-automation/lk-extraction/blob/fitriadc-dev/Main.ipynb)

- Jalankan langkah berikut:
   - Masukkan daftar nama emiten, kuarta, dan tahun yang ingin diesktrak pada variabel list `all_companies`, `kuartal`, dan `tahun`.
   - Klik **Runtime > Run all** dan tunggu proses selesai.

### **Menambah Emiten Baru**

- Buat folder baru pada folder `IDX_CALK_FX_SB_82`dengan format `/{kode_emiten}`
- Tambahkan file pdf di dalam folder emiten tersebut dengan format `{kode_emiten}-{tahun}-{kuartal}`
- Lakukan langkah `Menambah Jenis Informasi Baru untuk Diekstrak` sesuai jenis informasi yang dibutuhkan dari emiten tersebut.

### **Menambah Jenis Informasi Baru untuk Sebuah Emiten**

- Tentukan pattern marker (start dan end) yang tertera dalam dokument pdf sesuai jenis informasi yang ingin diekstrak.
- Tambahkan marker tersebut di `marker_config.py`.

# Source Code Documentation

#### Setup & Inisialisasi Lingkungan

Bagian pertama memuat berbagai *library* yang dibutuhkan, baik untuk pemrosesan file PDF (`pdfplumber`, `PyMuPDF`), pengolahan data (`pandas`), maupun manajemen sistem (`os`, `json`, `time`).

In [ ]:
import os
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
!pip install PyMuPDF
import warnings, logging, re, os
!pip install pdfplumber
import pdfplumber
import pandas as pd
from google.colab import files
import time
import fitz  # PyMuPDF
from dataclasses import dataclass, field
from typing import Dict, Optional

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 412.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Catat waktu mulai
start_program_time = time.time()

Mounted at /content/drive


Pencatatan waktu program dijalankan dengan time.time(). Hal ini digunakan untuk mengetahui durasi total eksekusi program.

Selanjutnya, dilakukan konfigurasi path dasar sumber data dan folder output sebagai tempat menyimpan hasil ekstraksi. Jika belum tersedia, maka folder otomatis dibuat menggunakan `os.makedirs`.

In [ ]:
# Path dasar input dan output
base_path = "/content/drive/MyDrive/IDX_CALK_FX_SB_82"
output_path = os.path.join(base_path, "output")
os.makedirs(output_path, exist_ok=True)

# Konfigurasi nama file output
output_json_file = "hasil_semua.json"
output_file = "hasil_ekstraksi.xlsx"
benchmark_file = "benchmark_waktu.xlsx"

#### Parameter Pemrosesan & Multi-threading

Daftar perusahaan yang akan diproses disimpan dalam `all_companies`, serta tahun (`years`) dan kuartal (`quarters`) yang diinginkan. Pemrosesan dilakukan secara paralel (multi-threading) dengan batas maksimal thread (`max_workers`) yang disesuaikan dengan jumlah inti CPU.

In [ ]:
# Daftar perusahaan
# all_companies = [
#     "TLKM", "INDF", "INKP", "ADRO", "UNTR", "JSMR", "ICBP", "MEDC", "ISAT", "GIAA",
#     "PGAS", "EXCL", "TPIA", "SMGR", "MDKA", "BHIT", "BSDE", "WIKA", "IMAS", "TKIM",
#     "HMSP", "LPKR", "INDY", "BYAN", "INCO", "TBIG", "CTRA", "FREN", "EMTK", "CPIN",
#     "ANTM", "SMAR", "PTBA", "ADHI", "PANI", "HRUM", "AMRT", "BMTR", "SIMP", "JPFA",
#     "ITMG", "PWON", "ABMM", "AKRA", "AALI", "INTP", "KLBF", "MAPI", "BIPI", "APLN",
#     "MYOR", "DOID", "TBLA", "BUKA", "MNCN", "ENRG", "ASRI", "CMNP", "SMCB", "DNET",
#     "AUTO", "BKSL", "CENT", "GEMS", "SMDR", "GJTL", "UNVR", "BRMS", "CMNT", "KAEF",
#     "DSNG", "DUTI", "TOBA", "DILD", "TAPG", "MDLN", "NIRO", "JRPT", "PSAB", "KIJA",
#     "TINS", "LSIP", "ARCI", "MLPL", "FASW", "INDR", "PLIN", "MCOL", "TSPC", "SSMS",
#     "CSAP", "MAPA", "PTRO", "EPMT", "SILO", "SCMA", "LPCK", "MTDL", "ELSA", "SGRO"
# ]

all_companies = [ "ICBP","IMAS","TKIM", "BYAN"]

# all_companies = [
#     "INDF", "INKP", "ADRO", "UNTR", "JSMR", "ICBP", "MEDC", "ISAT", "GIAA", "PGAS",
#     "EXCL", "TPIA", "SMGR", "MDKA", "BHIT", "BSDE", "WIKA", "IMAS", "TKIM", "LPKR",
#     "BYAN", "INCO", "TBIG", "FREN", "EMTK", "CPIN", "ANTM", "PTBA", "ADHI", "HRUM",
#     "SIMP", "ABMM", "AKRA", "MAPI", "APLN", "MYOR", "DOID", "TBLA", "BUKA", "MNCN",
#     "ENRG", "DNET", "AUTO", "BKSL", "CENT", "GEMS", "SMDR", "GJTL", "BRMS", "DSNG",
#     "DUTI", "TOBA", "DILD", "TAPG", "NIRO", "JRPT", "PSAB", "KIJA", "TINS", "FASW",
#     "MCOL", "TSPC", "SSMS", "CSAP", "MAPA", "PTRO", "EPMT", "SILO", "SCMA", "MTDL",
#     "ELSA", "SGRO"
# ]

all_companies = [
    "CMNP",
    "SMCB",
    "UNVR",
    "CMNT",
    "KAEF",
    "MDLN",
    "LSIP",
    "ARCI",
    "MLPL",
    "INDR",
    "PLIN",
    "LPCK"
]



# Tahun dan kuartal yang ingin diproses
years = ["2023", "2024"]
quarters = ["Q1", "Q2", "Q3"]
max_workers = min(10, os.cpu_count() or 4)
batch_size = 10

import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

#### Konfigurasi Marker untuk Ekstraksi Teks

Marker adalah kata kunci yang digunakan untuk menemukan bagian penting dari laporan keuangan. File `marker_config.py` diunduh langsung dari GitHub dan berisi konfigurasi penanda untuk berbagai perusahaan. Selanjutnya diambil konfigurasi marker untuk ratusan emiten saham dari file `marker_config.py`yang berisi aturan atau pola teks spesifik yang digunakan untuk mengekstraksi informasi tertentu dari laporan keuangan PDF emiten terkait. Setelah semua konfigurasi diimpor, kode tersebut membuat sebuah dictionary bernama marker_config_map yang memetakan ticker saham ke konfigurasi marker yang sesuai.

In [ ]:
!wget https://raw.githubusercontent.com/cbi-automation/lk-extraction/fitriadc-dev/marker_config.py -O marker_config.py


from marker_config import (
    ELSA_marker_config,
    SGRO_marker_config,
    TLKM_marker_config,
    INDF_marker_config,
    INKP_marker_config,
    ADRO_marker_config,
    UNTR_marker_config,
    JSMR_marker_config,
    ICBP_marker_config,
    MEDC_marker_config,
    ISAT_marker_config,
    GIAA_marker_config,
    PGAS_marker_config,
    EXCL_marker_config,
    TPIA_marker_config,
    SMGR_marker_config,
    MDKA_marker_config,
    TOWR_marker_config,
    BHIT_marker_config,
    BSDE_marker_config,
    WIKA_marker_config,
    IMAS_marker_config,
    TKIM_marker_config,
    PTPP_marker_config,
    HMSP_marker_config,
    LPKR_marker_config,
    DSSA_marker_config,
    INDY_marker_config,
    BYAN_marker_config,
    INCO_marker_config,
    TBIG_marker_config,
    CTRA_marker_config,
    FREN_marker_config,
    EMTK_marker_config,
    KRAS_marker_config,
    CPIN_marker_config,
    ANTM_marker_config,
    SMAR_marker_config,
    PTBA_marker_config,
    ADHI_marker_config,
    PANI_marker_config,
    HRUM_marker_config,
    AMRT_marker_config,
    BMTR_marker_config,
    SIMP_marker_config,
    JPFA_marker_config,
    ITMG_marker_config,
    KPIG_marker_config,
    PWON_marker_config,
    ABMM_marker_config,
    SMRA_marker_config,
    AKRA_marker_config,
    AALI_marker_config,
    INTP_marker_config,
    KLBF_marker_config,
    MAPI_marker_config,
    BIPI_marker_config,
    APLN_marker_config,
    MYOR_marker_config,
    DOID_marker_config,
    TBLA_marker_config,
    BUKA_marker_config,
    MNCN_marker_config,
    ERAA_marker_config,
    ENRG_marker_config,
    ASRI_marker_config,
    CMNP_marker_config,
    SMCB_marker_config,
    POWR_marker_config,
    DNET_marker_config,
    AUTO_marker_config,
    BKSL_marker_config,
    CENT_marker_config,
    GEMS_marker_config,
    SMDR_marker_config,
    GJTL_marker_config,
    PPRO_marker_config,
    # GGRP_marker_config,
    UNVR_marker_config,
    BRMS_marker_config,
    CMNT_marker_config,
    KAEF_marker_config,
    DSNG_marker_config,
    DUTI_marker_config,
    TOBA_marker_config,
    DILD_marker_config,
    TAPG_marker_config,
    MDLN_marker_config,
    NIRO_marker_config,
    JRPT_marker_config,
    PSAB_marker_config,
    KIJA_marker_config,
    TINS_marker_config,
    LSIP_marker_config,
    ARCI_marker_config,
    MLPL_marker_config,
    FASW_marker_config,
    INDR_marker_config,
    PLIN_marker_config,
    MCOL_marker_config,
    TSPC_marker_config,
    SSMS_marker_config,
    CSAP_marker_config,
    MAPA_marker_config,
    PTRO_marker_config,
    EPMT_marker_config,
    SILO_marker_config,
    SCMA_marker_config,
    LPCK_marker_config,
    MTDL_marker_config
)

marker_config_map = {
    "ELSA":ELSA_marker_config,
    "SGRO": SGRO_marker_config,
    "TLKM": TLKM_marker_config,
    "INDF": INDF_marker_config,
    "INKP": INKP_marker_config,
    "ADRO": ADRO_marker_config,
    "UNTR": UNTR_marker_config,
    "JSMR": JSMR_marker_config,
    "ICBP": ICBP_marker_config,
    "MEDC": MEDC_marker_config,
    "ISAT": ISAT_marker_config,
    "GIAA": GIAA_marker_config,
    "PGAS": PGAS_marker_config,
    "EXCL": EXCL_marker_config,
    "TPIA": TPIA_marker_config,
    "SMGR": SMGR_marker_config,
    "MDKA": MDKA_marker_config,
    "TOWR": TOWR_marker_config,
    "BHIT": BHIT_marker_config,
    "BSDE": BSDE_marker_config,
    "WIKA": WIKA_marker_config,
    "IMAS": IMAS_marker_config,
    "TKIM": TKIM_marker_config,
    "PTPP": PTPP_marker_config,
    "HMSP": HMSP_marker_config,
    "LPKR": LPKR_marker_config,
    "DSSA": DSSA_marker_config,
    "INDY": INDY_marker_config,
    "BYAN": BYAN_marker_config,
    "INCO": INCO_marker_config,
    "TBIG": TBIG_marker_config,
    "CTRA": CTRA_marker_config,
    "FREN": FREN_marker_config,
    "EMTK": EMTK_marker_config,
    "KRAS": KRAS_marker_config,
    "CPIN": CPIN_marker_config,
    "ANTM": ANTM_marker_config,
    "SMAR": SMAR_marker_config,
    "PTBA": PTBA_marker_config,
    "ADHI": ADHI_marker_config,
    "PANI": PANI_marker_config,
    "HRUM": HRUM_marker_config,
    "AMRT": AMRT_marker_config,
    "BMTR": BMTR_marker_config,
    "SIMP": SIMP_marker_config,
    "JPFA": JPFA_marker_config,
    "ITMG": ITMG_marker_config,
    "KPIG": KPIG_marker_config,
    "PWON": PWON_marker_config,
    "ABMM": ABMM_marker_config,
    "SMRA": SMRA_marker_config,
    "AKRA": AKRA_marker_config,
    "AALI": AALI_marker_config,
    "INTP": INTP_marker_config,
    "KLBF": KLBF_marker_config,
    "MAPI": MAPI_marker_config,
    "BIPI": BIPI_marker_config,
    "APLN": APLN_marker_config,
    "MYOR": MYOR_marker_config,
    "DOID": DOID_marker_config,
    "TBLA": TBLA_marker_config,
    "BUKA": BUKA_marker_config,
    "MNCN": MNCN_marker_config,
    "ERAA": ERAA_marker_config,
    "ENRG": ENRG_marker_config,
    "ASRI": ASRI_marker_config,
    "CMNP": CMNP_marker_config,
    "SMCB": SMCB_marker_config,
    "POWR": POWR_marker_config,
    "DNET": DNET_marker_config,
    "AUTO": AUTO_marker_config,
    "BKSL": BKSL_marker_config,
    "CENT": CENT_marker_config,
    "GEMS": GEMS_marker_config,
    "SMDR": SMDR_marker_config,
    "GJTL": GJTL_marker_config,
    "PPRO": PPRO_marker_config,
    # "GGRP": GGRP_marker_config,
    "UNVR": UNVR_marker_config,
    "BRMS": BRMS_marker_config,
    "CMNT": CMNT_marker_config,
    "KAEF": KAEF_marker_config,
    "DSNG": DSNG_marker_config,
    "DUTI": DUTI_marker_config,
    "TOBA": TOBA_marker_config,
    "DILD": DILD_marker_config,
    "TAPG": TAPG_marker_config,
    "MDLN": MDLN_marker_config,
    "NIRO": NIRO_marker_config,
    "JRPT": JRPT_marker_config,
    "PSAB": PSAB_marker_config,
    "KIJA": KIJA_marker_config,
    "TINS": TINS_marker_config,
    "LSIP": LSIP_marker_config,
    "ARCI": ARCI_marker_config,
    "MLPL": MLPL_marker_config,
    "FASW": FASW_marker_config,
    "INDR": INDR_marker_config,
    "PLIN": PLIN_marker_config,
    "MCOL": MCOL_marker_config,
    "TSPC": TSPC_marker_config,
    "SSMS": SSMS_marker_config,
    "CSAP": CSAP_marker_config,
    "MAPA": MAPA_marker_config,
    "PTRO": PTRO_marker_config,
    "EPMT": EPMT_marker_config,
    "SILO": SILO_marker_config,
    "SCMA": SCMA_marker_config,
    "LPCK": LPCK_marker_config,
    "MTDL": MTDL_marker_config
}

--2025-06-11 06:26:14--  https://raw.githubusercontent.com/cbi-automation/lk-extraction/fitriadc-dev/marker_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23469 (23K) [text/plain]
Saving to: ‘marker_config.py’

marker_config.py    100%[===================>]  22.92K  --.-KB/s    in 0.003s  

2025-06-11 06:26:14 (8.12 MB/s) - ‘marker_config.py’ saved [23469/23469]



#### Ekstraksi Informasi & Parsing Dokumen

Fungsi `extract_text()` bertanggung jawab untuk membaca seluruh isi teks dari setiap halaman PDF. Library fitz (alias PyMuPDF) digunakan karena lebih cepat dan andal dibanding library lain seperti pdfminer.

In [ ]:
# Fungsi ekstraksi teks yang cepat
def extract_text(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

Program menggunakan Python `@dataclass` untuk menyimpan hasil ekstraksi dalam bentuk struktur data Company. Ini memudahkan penyimpanan dan konversi ke format tabular seperti CSV atau Excel. Setiap objek `Company` merepresentasikan satu laporan keuangan dari satu perusahaan pada satu kuartal.

In [ ]:
@dataclass
class Company:
    kuartal: str = "-"
    perusahaan: str = "-"
    satuan: str = "-"
    risiko_nilai_tukar: str = "-"

field_map: Dict[str, str] = {
    "kuartal": "Kuartal",
    "perusahaan": "Perusahaan",
    "satuan": "Disajikan dalam",
    "risiko_nilai_tukar": "Efek Risiko Nilai Tukar",
    "risiko_suku_bunga": "Efek Risiko Suku Bunga",
}

def get_str(value) -> str:
    if value is None:
        return "-"
    val = str(value).strip()
    return val if val else "-"

def generate_company(doc: Optional[dict]) -> Company:
    data = {}
    for attr, source_key in field_map.items():
        data[attr] = get_str(doc.get(source_key)) if doc else "-"
    return Company(**data)

company = Company()

`field_map` adalah dictionary yang berfungsi untuk memetakan nama atribut internal (misalnya risiko_suku_bunga) ke nama kolom hasil ekstraksi yang lebih manusiawi (seperti "Efek Risiko Suku Bunga"). Hal ini membuat hasil akhir lebih mudah dibaca dan diekspor ke Excel atau antarmuka lain.

Fungsi `get_str()` memastikan bahwa nilai dari hasil ekstraksi selalu berupa string yang bersih dari spasi atau None. Jika nilai kosong atau tidak tersedia, fungsi ini akan mengembalikan tanda -, agar hasil tetap konsisten dan tidak menimbulkan error saat diproses lebih lanjut.

Fungsi `generate_company()` membentuk satu instance Company berdasarkan hasil ekstraksi teks PDF dan marker yang sesuai untuk masing-masing perusahaan. Ia memanggil fungsi-fungsi find_satuan, find_nilai_tukar, dan find_suku_bunga dari konfigurasi marker emiten.

In [ ]:

def find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal="2022"):
    start_marker, end_marker = marker_pairs[0]
    start_idx = text.find(start_marker)
    if start_idx == -1:
        print(f"[❗] Start marker tidak ditemukan di {kuartal}: {start_marker}")
        return "-"  # Jika tidak ditemukan, kembalikan "-" sebagai indikator tidak ditemukan

    search_range = text[start_idx:]
    end_relative = search_range.find(end_marker)
    if end_relative == -1:
        print(f"[❗] End marker tidak ditemukan setelah start marker di {kuartal}: {end_marker}")
        return "-"  # Jika tidak ditemukan, kembalikan "-" sebagai indikator tidak ditemukan

    end_idx = start_idx + end_relative

    orig_start_idx = text.lower().find(start_marker.lower())
    orig_end_idx = text.lower().find(end_marker.lower(), orig_start_idx)

    if orig_start_idx != -1 and orig_end_idx != -1:
        content_raw = text[orig_start_idx + len(start_marker): orig_end_idx]
        snippet = content_raw.strip()
    else:
        snippet = text[start_idx + len(start_marker): end_idx].strip()

    return snippet  # Kembalikan string langsung, bukan list atau dict

Fungsi `process_pdf_batch()` menangani pemrosesan banyak file PDF secara paralel dalam batch. Ia akan mencoba mengimpor konfigurasi marker perusahaan secara dinamis, mengeksekusi generate_company, dan menyimpan hasil ekstraksi ke list `results`.

In [ ]:
def find_nilai_tukar(text, marker_pairs, company: Company, kuartal: str):
    # Gabungkan newline jadi spasi agar regex lebih fleksibel
    teks_kotor = find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal)

    # Cek apakah teks_kotor adalah list, dan gabungkan menjadi string jika iya
    if isinstance(teks_kotor, list):
        teks_kotor = " ".join(teks_kotor)

    teks_bersih = teks_kotor.replace("\n", " ")

    company.risiko_nilai_tukar = teks_bersih

def find_suku_bunga(text, marker_pairs, company: Company, kuartal: str):
    # Gabungkan newline jadi spasi agar regex lebih fleksibel
    teks_kotor = find_paragraphs_by_marker_pairs(text, marker_pairs, kuartal)

    # Cek apakah teks_kotor adalah list, dan gabungkan menjadi string jika iya
    if isinstance(teks_kotor, list):
        teks_kotor = " ".join(teks_kotor)

    teks_bersih = teks_kotor.replace("\n", " ")

    company.risiko_suku_bunga = teks_bersih

Fungsi `find_nilai_tukar` dan `find_suku_bunga` bertanggung jawab untuk mengekstrak teks tertentu berdasarkan marker yang diberikan, yaitu terkait dengan risiko nilai tukar dan suku bunga. Keduanya menggunakan `find_paragraphs_by_marker_pairs` untuk mencari paragraf yang relevan dalam teks berdasarkan pasangan marker yang diberikan.

In [ ]:
def process_all_markers(text, kuartal, emiten):
    company = Company()
    company.perusahaan = emiten
    company.kuartal = kuartal

    markers = marker_config_map.get(emiten.upper())

    for marker_name, marker_pairs in markers.items():
        function_name = marker_to_function.get(marker_name)
        if not function_name:
            continue

        func = globals().get(function_name)
        if not func:
            print(f"[⚠️] Fungsi '{function_name}' tidak ditemukan.")
            continue

        # ⬇️ Fungsi sekarang juga menerima objek company yang akan di-update
        func(text, marker_pairs, company, kuartal)

    return company

# Mapping marker ke fungsi
marker_to_function = {
    "marker1": "find_satuan",
    "marker2": "find_nilai_tukar",
    "marker3": "find_suku_bunga"
}

def find_satuan(text, marker_pairs, company: Company, kuartal: str):
    results = find_paragraphs_by_marker_pairs(text, marker_pairs,kuartal)
    # satuan = results if results else "-"
    company.satuan = results

Fungsi `process_all_markers` menerima teks laporan, kuartal, dan nama emiten, lalu mengiterasi markers yang relevan untuk emiten tersebut. Setiap marker dipetakan ke fungsi yang sesuai menggunakan `marker_to_function`, dan kemudian fungsi itu dipanggil untuk mengekstrak data yang relevan. Fungsi-fungsi yang dipanggil akan memperbarui objek `Company` dengan hasil ekstraksi.

In [ ]:
# Fungsi pemrosesan individual
def process_file(emiten, year, quarter):
    kuartal_label = f"{year}-{quarter}"
    pdf_file = f"{base_path}/{emiten}/{emiten}-{year}-{quarter}.pdf"
    if not os.path.exists(pdf_file):
        print(f"[⚠️] File tidak ditemukan: {pdf_file}")
        return None

    print(f"\n🔍 Memproses: {emiten} - {kuartal_label}")
    try:
        start_time = time.time()
        text = extract_text(pdf_file)
        hasil = process_all_markers(text, kuartal_label, emiten)
        elapsed = time.time() - start_time
        print(f"\n✅ {kuartal_label}-{emiten} berhasil diproses dalam {elapsed:.2f} detik.")
        return f"{emiten}-{kuartal_label}", hasil.__dict__
    except Exception as e:
        print(f"[⚠️] Error pada {emiten}-{kuartal_label}: {e}")
        return None

Fungsi `process_file` menerima informasi emiten, tahun, dan kuartal, serta nama file PDF yang relevan. Jika file ditemukan, teks diekstraksi menggunakan `extract_text`, kemudian diproses dengan memanggil process_all_markers untuk mengekstrak data berdasarkan marker. Durasi pemrosesan dihitung dan dicatat

Dilakukan pemeriksaan terlebih dahulu, apakah file JSON hasil proses sebelumnya `(output_json_file`) ada. Jika file ada, maka isinya akan dimuat ke dalam variabel all_results_dicts_dinamis. Jika file tidak ada, variabel ini diinisialisasi sebagai dictionary kosong. Ini memastikan bahwa data yang sudah diproses sebelumnya tidak diproses ulang.

In [ ]:
if os.path.exists(output_json_file):
    with open(output_json_file, "r", encoding="utf-8") as f:
        all_results_dicts_dinamis = json.load(f)
else:
    all_results_dicts_dinamis = {}

# Simpan waktu proses per kuartal
benchmark_data = []

# Cek apakah sudah diproses
def is_processed(emiten, year, quarter):
    key = f"{emiten}-{year}-{quarter}"
    return key in all_results_dicts_dinamis


Fungsi `process_batch` memproses satu batch emiten dalam satu kuartal. Untuk setiap emiten dalam batch, fungsi ini memeriksa apakah laporan sudah diproses. Jika belum, fungsi ini memanggil `process_file` untuk memproses file laporan keuangan emiten tersebut. Setelah pemrosesan selesai, waktu yang digunakan untuk memproses laporan dihitung dan hasilnya disimpan dalam `batch_results`.

In [ ]:
def process_batch(batch_emiten, year, quarter):
    batch_results = []
    for emiten in batch_emiten:
        if not is_processed(emiten, year, quarter):
            start_time = time.time()
            result = process_file(emiten, year, quarter)
            if result:
                key, data = result
                batch_results.append((key, data, time.time() - start_time))
    return batch_results

In [ ]:
if os.path.exists(output_json_file):
    with open(output_json_file, "r", encoding="utf-8") as f:
        all_results_dicts_dinamis = json.load(f)
else:
    all_results_dicts_dinamis = {}

# Simpan waktu proses per kuartal
benchmark_data = []

# Cek apakah sudah diproses
def is_processed(emiten, year, quarter):
    key = f"{emiten}-{year}-{quarter}"
    return key in all_results_dicts_dinamis

Daftar emiten dibagi menjadi beberapa batch berdasarkan ukuran batch '(batch_size)'. Ini membantu untuk mengelompokkan emiten dalam jumlah yang lebih kecil sehingga proses dapat dijalankan secara paralel dan lebih efisien.

In [ ]:
# Bagi jadi batch
emiten_batches = [all_companies[i:i + batch_size] for i in range(0, len(all_companies), batch_size)]

Untuk setiap batch emiten, kode menggunakan `ThreadPoolExecutor` untuk menjalankan proses pemrosesan laporan secara paralel. Setiap kuartal dan tahun diproses secara bersamaan untuk setiap emiten dalam batch. `max_workers ` menentukan jumlah thread yang akan digunakan secara bersamaan untuk pemrosesan ini.

In [ ]:
# Jalankan per batch per kuartal
for batch_index, batch_emiten in enumerate(emiten_batches, start=1):
    print(f"\n🚀 Mulai Batch {batch_index}: {batch_emiten}")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_batch, batch_emiten, year, quarter)
            for year in years
            for quarter in quarters
        ]

        for future in as_completed(futures):
            batch_results = future.result()
            for key, data, durasi in batch_results:
                all_results_dicts_dinamis[key] = data
                emiten, year, quarter = key.split("-")
                benchmark_data.append({
                    "Emiten": emiten,
                    "Kuartal": f"{year}-{quarter}",
                    "Durasi (detik)": round(durasi, 2)
                })

                # Simpan progres JSON setiap selesai 1 kuartal
                with open(output_json_file, "w", encoding="utf-8") as f:
                    json.dump(all_results_dicts_dinamis, f, indent=2, ensure_ascii=False)



🚀 Mulai Batch 1: ['CMNP', 'SMCB', 'UNVR', 'CMNT', 'KAEF', 'MDLN', 'LSIP', 'ARCI', 'MLPL', 'INDR']
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/CMNP/CMNP-2023-Q1.pdf

🔍 Memproses: CMNP - 2023-Q2
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/SMCB/SMCB-2023-Q1.pdf

🔍 Memproses: UNVR - 2023-Q1

✅ 2023-Q1-UNVR berhasil diproses dalam 5.38 detik.

🔍 Memproses: CMNT - 2023-Q1

✅ 2023-Q2-CMNP berhasil diproses dalam 9.77 detik.

🔍 Memproses: SMCB - 2023-Q2

✅ 2023-Q1-CMNT berhasil diproses dalam 7.70 detik.

✅ 2023-Q2-SMCB berhasil diproses dalam 4.06 detik.
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/KAEF/KAEF-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/MDLN/MDLN-2023-Q1.pdf

🔍 Memproses: UNVR - 2023-Q2
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/LSIP/LSIP-2023-Q1.pdf
[⚠️] File tidak ditemukan: /content/drive/MyDrive/IDX_CALK_FX_SB_82/ARCI/ARCI-2023-Q1.pdf
[⚠️] Fil

Setelah batch selesai diproses, hasil dari setiap `future` yang disubmit akan diambil. Untuk setiap hasil (`key`, `data`, dan `durasi)`, data tersebut disimpan ke dalam `all_results_dicts_dinamis`. Durasi pemrosesan per kuartal juga dicatat dalam benchmark_data.

Setelah pemrosesan satu kuartal selesai, hasilnya disimpan dalam file JSON (`output_json_file`). Ini memastikan bahwa data yang sudah diproses tidak hilang jika program terhenti di tengah jalan.

#### Output & Evaluasi Kinerja

Data yang terkumpul dalam `all_results_dicts_dinamis` diubah menjadi sebuah DataFrame menggunakan pandas. Data ini kemudian disortir berdasarkan kolom Emiten dan Kuartal. Begitu juga dengan data benchmark (`benchmark_data`), yang Setelah data disortir, hasilnya disimpan dalam file Excel untuk data utama (`output_file)` dan file benchmark (`benchmark_file`) juga diubah menjadi DataFrame dan disortir berdasarkan Emiten dan Kuartal.

In [ ]:
import re

df = pd.DataFrame(list(all_results_dicts_dinamis.values()))
df.to_excel(output_file, index=False)
print(f"\n✅ File hasil utama: {output_file}")


✅ File hasil utama: hasil_ekstraksi.xlsx


Waktu total yang dibutuhkan untuk menjalankan program dihitung dan dicetak setelah semua batch selesai diproses. Jika kode dijalankan di Google Colab, file hasil (`output_file` dan `benchmark_file`) akan diunduh secara otomatis ke mesin lokal pengguna.

In [ ]:
# Urutkan benchmark berdasarkan 'Emiten' dan 'Kuartal'
benchmark_df = pd.DataFrame(benchmark_data)
# benchmark_df.sort_values(by=["Emiten", "Kuartal"], inplace=True)

print(f"\n✅ File hasil utama: {output_file}")
benchmark_df.to_excel(benchmark_file, index=False)


print(f"⏱️ File benchmark waktu dibuat: {benchmark_file}")

# Waktu total program
end_program_time = time.time()
print(f"\n🕒 Total waktu runtime program: {round(end_program_time - start_program_time, 2)} detik")

# Auto-download jika di Colab
try:
    from google.colab import files
    files.download(output_file)
    files.download(benchmark_file)
except:
    pass


✅ File hasil utama: hasil_ekstraksi.xlsx
⏱️ File benchmark waktu dibuat: benchmark_waktu.xlsx

🕒 Total waktu runtime program: 166.35 detik


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Process Documentation

## 1. CAPTURE – Menentukan Kebutuhan dan Target Informasi


**Tujuan:**  
Memastikan informasi yang dibutuhkan untuk analisis dan pemantauan sektor korporasi.

**Proses:**
- Identifikasi kebutuhan informasi kebijakan yang ingin diekstrak dari laporan keuangan, misalnya: risiko nilai tukar, suku bunga.
- Pengumpulan dokumen laporan keuangan dari Web IDX Indonesia.
- Penetapan marker dan keyword pencarian, seperti: `nilai tukar`, `mata uang asing`, dan sebagainya.
- Pembacaan PDF halaman demi halaman.
- Pencocokan marker berdasarkan konfigurasi per perusahaan untuk menemukan paragraf relevan.
- Ekstraksi paragraf yang mengandung informasi sesuai marker.

## 2. COMPILE – Ekstraksi dan Strukturisasi Data Otomatis

**Tujuan:**  
Mengubah dokumen unstructured menjadi data terstruktur dan siap dianalisis.

**Proses:**
- Praproses teks: normalisasi whitespace, penghapusan header/footer, dan filter karakter asing.
- Identifikasi entitas penting seperti nama mata uang, nilai eksposur, dan instrumen lindung nilai.
- Strukturisasi hasil ekstraksi ke dalam format dictionary atau JSON.

## 3. ANALYSIS – Pemanfaatan Data untuk Insight dan Kebijakan

**Tujuan:**  
Menyajikan insight relevan, cepat, dan konsisten untuk mendukung keputusan makroprudensial.

**Proses:**
- Agregasi data antar perusahaan dan kuartal untuk mengidentifikasi tren sektoral.
- Pemetaan eksposur nilai tukar berdasarkan sektor, industri, atau perusahaan besar.
- Visualisasi data dan pengembangan dashboard monitoring.
- Penyusunan narasi untuk CHAF atau Kajian Stabilitas Keuangan (KSK).
- Penyampaian rekomendasi kebijakan berbasis evidence kepada stakeholder internal atau eksternal.